<a href="https://colab.research.google.com/github/hatefap/CrackingTheInterview/blob/master/mbert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [ ]:
farsi_path = "drive/MyDrive/final_final.csv"
english_path = "drive/MyDrive/my_final_english_hate.csv"
eng = pd.read_csv(english_path)
farsi = pd.read_csv(farsi_path)
farsi.head()

,id,text,ref1,ref2,link,norm_text,google,mbart,ref3
0,0,بچه های عزیز این کشتزار منو بلاک کرده یکی که د...,1.0,0.0,https://twitter.com/DrErovay/status/1249849840...,بچه‌های عزیز این کشتزار منو بلاک کرده یکی که د...,Children's Menu block the field of view of one...,"My dear children, I've been blocked on this fa...",1.0
1,1,سیاسیون محترم از نمایندگان گرفته تا شهردارا...,0.0,0.0,https://twitter.com/atefeh heidari/status/1007...,سیاسیون محترم از نمایندگان گرفته تا شهرداران ...,Distinguished delegates politicians to munici...,From respectable politicians to elected offici...,0.0
2,2,با اعدام مخالفم \n\nکاش راهکاری برای نابودی و ...,1.0,0.0,https://twitter.com/fariborz c/status/10124857...,با اعدام مخالفم \n\nکاش راهکاری برای نابودی و ...,I disagree with execution\n\nIf only way to an...,"With the execution of my opponent, I wish ther...",1.0
3,3,دونوع آدم داریم : یکی اونکه هیچی به کیرش نیست ...,1.0,0.0,https://twitter.com/llil ili lill/status/10190...,دونوع آدم داریم: یکی اونکه هیچی به کیرش نیست و...,There are two types of people: one Avnkh nothi...,There are two types of people: There's the one...,0.0
4,4,هيچ زنداني بدتر از ،دختر بودن، نيست....,1.0,0.0,https://twitter.com/virgoOoOol/status/10233572...,هیچ زندانی بدتر از، دختر بودن، نیست ….,"No prisoners worse, the girl is not ....",There is no prison worse than being a girl...,0.0


In [ ]:
eng.head()

,Unnamed: 0,tweet,class
0,0,"""@USER @USER @USER queer"" gaywad",1
1,1,"""@USER @USER alsarabsss"" hes a beaner smh you ...",1
2,2,"""@USER @USER you're fucking gay, blacklisted h...",1
3,3,"""@USER LMFAOOOO I HATE BLACK PEOPLE https://t....",1
4,4,"""@USER ""At least I'm not a nigger"" http://t.co...",1


In [ ]:
eng = eng.drop("Unnamed: 0", axis=1)

In [ ]:
farsi = farsi[['norm_text', 'ref3']]
farsi.columns = ['tweet', 'class']
farsi.head()

,tweet,class
0,بچه‌های عزیز این کشتزار منو بلاک کرده یکی که د...,1.0
1,سیاسیون محترم از نمایندگان گرفته تا شهرداران ...,0.0
2,با اعدام مخالفم \n\nکاش راهکاری برای نابودی و ...,1.0
3,دونوع آدم داریم: یکی اونکه هیچی به کیرش نیست و...,0.0
4,هیچ زندانی بدتر از، دختر بودن، نیست ….,0.0


In [ ]:
farsi = farsi.dropna()
farsi.tail()

,tweet,class
5195,من ایشون میشناسم تو اینستام پیج داره تا جایی ک...,0.0
5196,پسرا، هم جنسا، چرا دختر هرچی بددهن باشه و کصشع...,0.0
5197,هدایت و بازداشت شده است. \nامارات سیاست سکوت ر...,0.0
5198,عزیزان، کسی گفته کاربران توئیتر مستجاب الدعوه‌...,0.0
5199,میگن هیچ چیزی بدون دلیل نیست\nهمه چی دست به دس...,0.0


In [ ]:
def deEmojify(row):
    text = row.tweet
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

In [ ]:
eng['tweet'] = eng.apply(deEmojify, axis=1)

In [ ]:
eng = eng.sample(frac=1).reset_index(drop=True)